In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import matplotlib.pyplot as plt
import tflearn
import tensorflow as tf

%matplotlib inline

In [2]:
def oneHotEncoder(label, n):
    """ One-Hot-Encoder for n class case """
    tmp = np.zeros((len(label), n))
    for number in range(n):
        tmp[:, number] = (label[:, 0] == number)
    tmp = tmp.astype(int)

    return tmp

In [3]:
# Data loading and preprocessing
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

# get detailed data
train_img = mnist.train.images
train_label = mnist.train.labels

cv_img = mnist.validation.images
cv_label = mnist.validation.labels

test_img = mnist.test.images
test_label = mnist.test.labels

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
with tf.Graph().as_default():
    tflearn.init_graph(num_cores=2, gpu_memory_fraction=0.4)
    
    # Building the encoder
    encoder = tflearn.input_data(shape=[None, 784])
    encoder = tflearn.fully_connected(encoder, 512, activation='sigmoid')
    encoder = tflearn.fully_connected(encoder, 256, activation='sigmoid')

    # Building the decoder
    decoder = tflearn.fully_connected(encoder, 512, activation='sigmoid')
    decoder = tflearn.fully_connected(decoder, 784, activation='sigmoid')

    # Regression, with mean square error
    net = tflearn.regression(decoder, optimizer='adam', learning_rate=0.001,
                             loss='mean_square', metric=None)

    # Training the auto encoder
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit(train_img, train_img, n_epoch=20, validation_set=(cv_img, cv_img),
              run_id="auto_encoder", batch_size=256)

    # New model, re-using the same session, for weights sharing
    encoding_model = tflearn.DNN(encoder, session=model.session)
    train_encoded = encoding_model.predict(train_img)
    cv_encoded = encoding_model.predict(cv_img)
    test_encoded = encoding_model.predict(test_img)

train_encoded = np.array(train_encoded)
cv_encoded = np.array(cv_encoded)
test_encoded = np.array(test_encoded)

Training Step: 4300  | total loss: 0.01354
| Adam | epoch: 020 | loss: 0.01354 | val_loss: 0.00938 -- iter: 55000/55000
Training Step: 4300  | total loss: 0.01354
| Adam | epoch: 020 | loss: 0.01354 | val_loss: 0.00938 -- iter: 55000/55000
--


In [6]:
# Classification using tflearn
with tf.Graph().as_default():
    tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.8)

    # build the one-layer fully connected neural network
    net = tflearn.input_data(shape=[None, 256])
    net = tflearn.fully_connected(net, 10, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

    # fit the model
    model = tflearn.DNN(net)
    model.fit(train_encoded, train_label, n_epoch=20, validation_set=(cv_encoded, cv_label))

    # predict on the training and test dataset
    train_prediction = np.array(model.predict(train_encoded))
    test_prediction = np.array(model.predict(test_encoded))

Training Step: 17200  | total loss: 0.44562
| Adam | epoch: 020 | loss: 0.44562 | val_loss: 0.21219 -- iter: 55000/55000
Training Step: 17200  | total loss: 0.44562
| Adam | epoch: 020 | loss: 0.44562 | val_loss: 0.21219 -- iter: 55000/55000
--


In [7]:
# get the label in single number case
train_true_label = np.argmax(train_label, axis=1)
test_true_label = np.argmax(test_label, axis=1)

# get the label
train_pred_label = np.argmax(train_prediction, axis=1)
test_pred_label = np.argmax(test_prediction, axis=1)

# calculate the training and testing accuracy
train_acc = np.sum(train_pred_label == train_true_label) / len(train_true_label)
test_acc = np.sum(test_pred_label == test_true_label) / len(test_true_label)
print('Training Accuract:', str(np.round(train_acc, 5)))
print('Testing Accuract:', str(np.round(test_acc, 5)))

Training Accuract: 0.93815
Testing Accuract: 0.941
